### Webscrapping

In [1]:
import time
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd

In [2]:
# setup
DRIVER_PATH = '/webdrivers/chromedriver.exe'
browser = webdriver.Chrome(executable_path=DRIVER_PATH)

# Get the the premier league website at a specific year
browser.get("https://www.premierleague.com/results?co=1&se=42&cl=-1")
time.sleep(2)

# Get the body of the site
element_body = browser.find_element_by_tag_name("body")
# Scroll
no_of_pagedowns = 50

# get the list of match id for stats data
match_id_lst = []

for go_down in range(50,0,-1):
    element_body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    
unordered_list_matchList = browser.find_elements_by_class_name("matchList")

for matchList_container in unordered_list_matchList:
    li_element = matchList_container.find_elements_by_class_name("matchFixtureContainer")
    for id_attr in li_element:
        match_id = id_attr.get_attribute("data-comp-match-item")
        match_id_lst.append(match_id)

In [10]:
# loop through match ids and open the actual match in details
all_stat_dict = {}

for match_id in match_id_lst:
    #full url
    browser.get("https://www.premierleague.com/match/" + match_id)
    time.sleep(1)
    tabs = browser.find_element_by_tag_name("body")
    stat_tab = tabs.find_element_by_css_selector("div.tabLinks.matchNav > .tabs ul li:nth-child(3)")
    stat_tab.click()
    time.sleep(1)

    stat_table = browser.find_element_by_class_name("matchCentreStatsContainer")
    table_len = len(stat_table.find_elements_by_xpath("./tr"))

    count = 1
    hometeam_name = browser.find_element_by_xpath("""//*[@id="mainContent"]/div/section[2]/div[2]/div/div[2]/section[3]/div[2]/div[2]/table/thead/tr/th[1]/a
    """).text
    awayteam_name = browser.find_element_by_xpath("""//*[@id="mainContent"]/div/section[2]/div[2]/div/div[2]/section[3]/div[2]/div[2]/table/thead/tr/th[3]/a""").text
    
    fullTime_score = browser.find_element_by_xpath("""//*[@id="mainContent"]/div/section[2]/div[2]/section/div[3]/div/div/div[1]/div[2]/div/div""").text.split('-')
    hometeam_FTS = fullTime_score[0]
    awayteam_FTS = fullTime_score[1]
    
    hometeam_pos='0'
    awayteam_pos='0'
    hometeam_sot='0'
    awayteam_sot='0'
    hometeam_shots='0'
    awayteam_shots='0',
    hometeam_touches='0'
    awayteam_touches='0'
    hometeam_tackles='0'
    awayteam_tackles='0'
    hometeam_clearances='0'
    awayteam_clearances='0'
    hometeam_corners='0'
    awayteam_corners='0'
    hometeam_offsides='0'
    awayteam_offsides='0'
    hometeam_yCards='0'
    awayteam_yCards='0'
    hometeam_rCards='0'
    awayteam_rCards='0'
    hometeam_fConceded='0'
    awayteam_fConceded='0'
#     hometeam_pos,awayteam_pos,hometeam_sot,awayteam_sot,hometeam_shots,awayteam_shots,
#     hometeam_touches,awayteam_touches,hometeam_tackles,awayteam_tackles,hometeam_clearances,
#     awayteam_clearances,hometeam_corners,awayteam_corners,hometeam_offsides,awayteam_offsides,
#     hometeam_yCards,awayteam_yCards,hometeam_rCards,awayteam_rCards,hometeam_fConceded,
#     awayteam_fConceded = '0','0','0','0','0','0','0','0','0','0','0','0','0','0','0',
#     '0','0','0','0','0','0','0'
    
    

    for stat_row in range(table_len):
        category = stat_table.find_element_by_xpath("""./tr[{}]/td[2]/p""".format(count))
        hometeam = stat_table.find_element_by_xpath("""./tr[{}]/td[1]/p""".format(count))
        awayteam = stat_table.find_element_by_xpath("""./tr[{}]/td[3]/p""".format(count))
        count +=1

        if "Possession" in category.text:
            hometeam_pos = hometeam.text
            awayteam_pos = awayteam.text
        elif "Shots on target" in category.text:
            hometeam_sot = hometeam.text
            awayteam_sot = awayteam.text
        elif "Shots" in category.text:
            hometeam_shots = hometeam.text
            awayteam_shots = awayteam.text
        elif "Touches" in category.text:
            hometeam_touches = hometeam.text
            awayteam_touches = awayteam.text
        elif "Passes" in category.text:
            hometeam_touches = hometeam.text
            awayteam_touches = awayteam.text
        elif "Tackles" in category.text:
            hometeam_tackles = hometeam.text
            awayteam_tackles = awayteam.text
        elif "Clearance" in category.text:
            hometeam_clearances = hometeam.text
            awayteam_clearances = awayteam.text
        elif "Corners" in category.text:
            hometeam_corners = hometeam.text
            awayteam_corners = awayteam.text
        elif "Offsides" in category.text:
            hometeam_offsides = hometeam.text
            awayteam_offsides = awayteam.text
        elif "Yellow cards" in category.text:
            hometeam_yCards = hometeam.text
            awayteam_yCards = awayteam.text
        elif "Red cards" in category.text:
            hometeam_rCards = hometeam.text
            awayteam_rCards = awayteam.text
        elif "Fouls conceded" in category.text:
            hometeam_fConceded = hometeam.text
            awayteam_fConceded = awayteam.text
        
        #dictionary of matches with details for a specific season
        all_stat_dict["Match"+str(match_id)] = {"Match_id":match_id,
                                                "Hometeam": hometeam_name,
                                                "Awayteam":awayteam_name,
                                                "Hometeam_Score":hometeam_FTS,
                                                "Awayteam_Score":awayteam_FTS,
                                                "Home_Possession":hometeam_pos,
                                                "Away_Possession":awayteam_pos,
                                                "Home_Shots_on_target":hometeam_sot,
                                                "Away_Shots_on_target":awayteam_sot,
                                                "Home_Total_Shots":hometeam_shots,
                                                "Away_Total_Shots":awayteam_shots,
                                                "Home_Touches":hometeam_touches,
                                                "Away_Touches":awayteam_touches,
                                                "Home_Passes":hometeam_touches,
                                                "Away_Pasees":awayteam_touches,
                                                "Home_Takles":hometeam_tackles,
                                                "Away_Takles":awayteam_tackles,
                                                "Home_Clearance":hometeam_clearances,
                                                "Away_Clearance":awayteam_clearances,
                                                "Home_Corners":hometeam_corners,
                                                "Away_Corners":awayteam_corners,
                                                "Home_Offsides":hometeam_offsides,
                                                "Away_Offsides":awayteam_offsides,
                                                "Home_Yellow_Cards":hometeam_yCards,
                                                "Away_Yellow_Cards":awayteam_yCards,
                                                "Home_Red_Cards":hometeam_rCards,
                                                "Away_Red_Cards":awayteam_rCards,
                                                "Home_Fouls_Conceded":hometeam_fConceded,
                                                "Away_Fouls_Conceded":awayteam_fConceded 
                                                }
print(all_stat_dict)

{'Match12494': {'Match_id': '12494', 'Hometeam': 'Manchester United', 'Awayteam': 'AFC Bournemouth', 'Hometeam_Score': '3', 'Awayteam_Score': '1', 'Home_Possession': '66.3', 'Away_Possession': '33.7', 'Home_Shots_on_target': '5', 'Away_Shots_on_target': '0', 'Home_Total_Shots': '12', 'Away_Total_Shots': '7', 'Home_Touches': '677', 'Away_Touches': '342', 'Home_Passes': '677', 'Away_Pasees': '342', 'Home_Takles': '17', 'Away_Takles': '16', 'Home_Clearance': '11', 'Away_Clearance': '10', 'Home_Corners': '7', 'Away_Corners': '1', 'Home_Offsides': '4', 'Away_Offsides': '2', 'Home_Yellow_Cards': '1', 'Away_Yellow_Cards': '0', 'Home_Red_Cards': '0', 'Away_Red_Cards': '0', 'Home_Fouls_Conceded': '9', 'Away_Fouls_Conceded': '8'}, 'Match12485': {'Match_id': '12485', 'Hometeam': 'Arsenal', 'Awayteam': 'Aston Villa', 'Hometeam_Score': '4', 'Awayteam_Score': '0', 'Home_Possession': '59.6', 'Away_Possession': '40.4', 'Home_Shots_on_target': '7', 'Away_Shots_on_target': '2', 'Home_Total_Shots': '16',

In [16]:
def write_to_csv(stat_dict):
    with open('2015-16Season.csv', 'w') as f:
        f.write("Match_ID" + "," + "Hometeam" + "," + "Awayteam" + "," + "Hometeam Score" + "," + "Awayteam Score" + "," + "Home_Possession" 
                + "," + "Away_Possession" + "," + "Home_Shots_on_target" + "," + "Away_Shots_on_target" 
                + "," + "Home_Total_Shots"+ "," + "Away_Total_Shots" + "," + "Home_Touches" 
                + "," + "Away_Touches" + "," + "Home_Passes" + "," + "Away_Pasees" + "," + "Home_Takles"
                + "," + "Away_Takles"+ "," + "Home_Clearance"+ "," + "Away_Clearance"
                + "," + "Home_Corners"+ "," + "Away_Corners"+ "," + "Home_Offsides"
                + "," + "Away_Offsides"+ "," + "Home_Yellow_Cards"+ "," + "Away_Yellow_Cards"
                + "," + "Home_Red_Cards"+ "," + "Away_Red_Cards"+ "," + "Home_Fouls_Conceded"
                + "," + "Away_Fouls_Conceded" + '\n')
        
        for i in stat_dict.keys():
            output_str = str(stat_dict[i]['Match_id']) + "," + str(stat_dict[i]['Hometeam'])+ "," + str(stat_dict[i]['Awayteam']) + "," + str(stat_dict[i]['Hometeam_Score']) + "," + str(stat_dict[i]['Awayteam_Score']) + "," + str(stat_dict[i]['Home_Possession'])+ "," + str(stat_dict[i]['Away_Possession']) + ", " + str(stat_dict[i]['Home_Shots_on_target'])+ "," + str(stat_dict[i]['Away_Shots_on_target']) + "," + str(stat_dict[i]['Home_Total_Shots']) + "," + str(stat_dict[i]['Away_Total_Shots'])+ "," + str(stat_dict[i]['Home_Touches'])+ "," + str(stat_dict[i]['Away_Touches'])+ "," + str(stat_dict[i]['Home_Passes']) + "," + str(stat_dict[i]['Away_Pasees'])+ "," + str(stat_dict[i]['Home_Takles'])+ "," + str(stat_dict[i]['Away_Takles'])+ "," + str(stat_dict[i]['Home_Clearance'])+ "," + str(stat_dict[i]['Away_Clearance'])+ "," + str(stat_dict[i]['Home_Corners'])+ "," + str(stat_dict[i]['Away_Corners'])+ "," + str(stat_dict[i]['Home_Offsides'])+ "," + str(stat_dict[i]['Away_Offsides'])+ "," + str(stat_dict[i]['Home_Yellow_Cards'])+ "," + str(stat_dict[i]['Away_Yellow_Cards'])+ "," + str(stat_dict[i]['Home_Red_Cards']) + "," + str(stat_dict[i]['Away_Red_Cards'])+ "," + str(stat_dict[i]['Home_Fouls_Conceded']) + "," + str(stat_dict[i]['Away_Fouls_Conceded']) + '\n'
            f.write(output_str)

In [11]:
df = pd.DataFrame.from_dict(all_stat_dict, orient='index')
df

,Match_id,Hometeam,Awayteam,Hometeam_Score,Awayteam_Score,Home_Possession,Away_Possession,Home_Shots_on_target,Away_Shots_on_target,Home_Total_Shots,...,Home_Corners,Away_Corners,Home_Offsides,Away_Offsides,Home_Yellow_Cards,Away_Yellow_Cards,Home_Red_Cards,Away_Red_Cards,Home_Fouls_Conceded,Away_Fouls_Conceded
Match12494,12494,Manchester United,AFC Bournemouth,3,1,66.3,33.7,5,0,12,...,7,1,4,2,1,0,0,0,9,8
Match12485,12485,Arsenal,Aston Villa,4,0,59.6,40.4,7,2,16,...,5,4,0,0,0,1,0,0,12,6
Match12486,12486,Chelsea,Leicester City,1,1,54,46,4,5,17,...,7,6,8,0,0,0,0,0,5,12
Match12487,12487,Everton,Norwich City,3,0,45.5,54.5,8,2,15,...,4,11,1,1,0,0,0,0,7,11
Match12488,12488,Newcastle United,Tottenham Hotspur,5,1,38.5,61.5,10,5,19,...,3,8,1,1,1,2,1,0,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Match12115,12115,AFC Bournemouth,Aston Villa,0,1,58.4,41.6,2,3,11,...,6,3,0,0,3,4,0,0,13,13
Match12117,12117,Everton,Watford,2,2,66.6,33.4,5,5,10,...,8,2,4,1,1,2,0,0,7,13
Match12118,12118,Leicester City,Sunderland,4,2,44,56,8,5,20,...,6,3,0,1,2,4,0,0,13,17
Match12120,12120,Norwich City,Crystal Palace,1,3,62.9,37.1,6,7,17,...,1,4,4,2,1,0,0,0,14,20


In [17]:
write_to_csv(all_stat_dict)